In [1]:
## imports
import pandas as pd
import numpy as np
import os
from eda import insufficient_but_starting_eda
import seaborn as sns

In [2]:
# Load the Data
comp = pd.read_csv('inputs/cust_supply_2019_2022.csv')
sp500 = pd.read_csv('inputs/sp500_2022.csv')
acct_raw = pd.read_csv("inputs/acct_data.csv")

# Merging the data

## Merging compustat & sp500

In [3]:
comp2 = comp
comp2 = comp2[comp2['cnms'] != 'Not Reported']
comp3 = comp2.dropna(subset=['salecs'])

# we checked to see if these companies were in sp500, they were not, so deleting them was ok
# compzzz = comp2[comp2['cnms'] == 'Not Reported']  
# compzzz = compzzz.rename(columns = {'cik': 'CIK'})
# testng_merge = compzzz.merge(sp500, on='CIK', how = 'inner', indicator=True)

In [4]:
comp3 = comp3.rename(columns = {'cik': 'CIK'})
merged = comp3.merge(sp500, on='CIK', how = 'inner')
merged

#merged['CIK'].nunique() #385 firms

,gvkey,cid,cnms,ctype,gareac,gareat,salecs,sid,stype,srcdate,...,cusip,CIK,sic,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,Founded
0,1075,22,Retail non-residential electric service,MARKET,NaN,NaN,1509.514,9,BUSSEG,2019-12-31,...,723484101,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985
1,1075,21,Retail residential electric service,MARKET,NaN,NaN,1761.122,9,BUSSEG,2019-12-31,...,723484101,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985
2,1075,23,Wholesale energy sales,MARKET,NaN,NaN,121.805,9,BUSSEG,2019-12-31,...,723484101,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985
3,1075,25,Other sources,MARKET,NaN,NaN,16.308,9,BUSSEG,2019-12-31,...,723484101,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985
4,1075,24,Transmission services for others,MARKET,NaN,NaN,62.460,9,BUSSEG,2019-12-31,...,723484101,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9104,316056,3,10 Customers,COMPANY,NaN,NaN,656.420,1,BUSSEG,2019-12-31,...,G0176J109,1579241.0,3420,ALLE,Allegion,Industrials,Building Products,"New York City, New York",2013-12-02,1908
9105,316056,8,Non-U.S.,GEOREG,OTHER,REG,814.400,1,BUSSEG,2020-12-31,...,G0176J109,1579241.0,3420,ALLE,Allegion,Industrials,Building Products,"New York City, New York",2013-12-02,1908
9106,316056,4,United States,GEOREG,USA,ISO,1905.500,1,BUSSEG,2020-12-31,...,G0176J109,1579241.0,3420,ALLE,Allegion,Industrials,Building Products,"New York City, New York",2013-12-02,1908
9107,316056,3,10 Customers,COMPANY,NaN,NaN,652.776,1,BUSSEG,2020-12-31,...,G0176J109,1579241.0,3420,ALLE,Allegion,Industrials,Building Products,"New York City, New York",2013-12-02,1908


In [5]:
#looking at the dates - see there is a filing in every month...
merged['date'] = pd.to_datetime(merged['srcdate'])
dates = merged.sort_values(by='srcdate')

In [6]:
start_date = '2020-01-01'
end_date = '2021-12-31'
filtered_df = merged.query('@start_date <= date <= @end_date')

# get the indices of the filtered dates
filtered_indices = filtered_df.index

# drop the filtered dates from the original dataframe
filtered_out_df = merged.drop(filtered_indices)

filtered_out_df
#filtered_out_df['CIK'].nunique() #355 values

,gvkey,cid,cnms,ctype,gareac,gareat,salecs,sid,stype,srcdate,...,CIK,sic,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,Founded,date
0,1075,22,Retail non-residential electric service,MARKET,NaN,NaN,1509.514,9,BUSSEG,2019-12-31,...,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985,2019-12-31
1,1075,21,Retail residential electric service,MARKET,NaN,NaN,1761.122,9,BUSSEG,2019-12-31,...,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985,2019-12-31
2,1075,23,Wholesale energy sales,MARKET,NaN,NaN,121.805,9,BUSSEG,2019-12-31,...,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985,2019-12-31
3,1075,25,Other sources,MARKET,NaN,NaN,16.308,9,BUSSEG,2019-12-31,...,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985,2019-12-31
4,1075,24,Transmission services for others,MARKET,NaN,NaN,62.460,9,BUSSEG,2019-12-31,...,764622.0,4911,PNW,Pinnacle West,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,1985,2019-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9062,189491,4,Treatment,MARKET,NaN,NaN,397.000,1,BUSSEG,2019-12-31,...,1524472.0,3561,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,2011,2019-12-31
9063,189491,3,Transport,MARKET,NaN,NaN,1780.000,1,BUSSEG,2019-12-31,...,1524472.0,3561,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,2011,2019-12-31
9102,316056,8,Non-U.S.,GEOREG,OTHER,REG,865.100,1,BUSSEG,2019-12-31,...,1579241.0,3420,ALLE,Allegion,Industrials,Building Products,"New York City, New York",2013-12-02,1908,2019-12-31
9103,316056,4,United States,GEOREG,USA,ISO,1988.900,1,BUSSEG,2019-12-31,...,1579241.0,3420,ALLE,Allegion,Industrials,Building Products,"New York City, New York",2013-12-02,1908,2019-12-31


In [7]:
filtered_out_df['fyear'] = pd.to_datetime(filtered_out_df['srcdate']).dt.year

In [8]:
# filtering out data that doesn't have BOTH 2019 & 2022
filtered = filtered_out_df.groupby('gvkey').filter(lambda x: x['fyear'].max() == 2022)
filtered.to_csv('test.csv')

# checking numbers
#filtered['gvkey'].nunique(), filtered_out_df['gvkey'].nunique() # (89, 355)


## Identifying the unique `gvkey` to obtain accounting data

In [9]:
listkeys = pd.DataFrame(filtered_out_df['gvkey'].unique())
listkeys.to_csv('inputs/listkeys.csv', index=False)

## Merging accounting data & sp500

In [10]:
listkeys2 = listkeys.rename(columns={0: 'gvkey'})

merged_acct_raw_keys = pd.merge(listkeys2,acct_raw,how = 'inner', on='gvkey')
merged_acct_raw_keys['gvkey'].nunique() # 354

354

In [11]:
acct_df = merged_acct_raw_keys.query('fyear == 2019 or fyear == 2022') #can add `fyear == 2021`
acct_df = acct_df[['gvkey', 'fyear', 'ap', 'at', 'capx', 'cogs', 'epsfx', 'gp', 'invt', 'ni', 'rect', 'sale']]

prev_row = acct_df.iloc[0]
for index, row in acct_df.iloc[1:].iterrows():
    if row.values[1] == 2019.0:
        prev_row = row
    elif row.values[1] == 2022.0:
        calc_row = (row - prev_row) / prev_row * 100
        calc_row.rename({k: f"calc_{k}" for k in calc_row.index}, inplace=True)
        acct_df.loc[index, calc_row.index] = calc_row

acct_df['gvkey'].nunique() #354

354

# The Final Dataset

In [12]:
# merge acct with filtered_out_df
# comp_final = filtered_out_df[['gvkey','cnms', 'ctype', 'salecs','srcdate', 'conm', 'tic', 'cusip', 'CIK', 'sic', 'Symbol','GICS Sector','fyear']]

# Select the desired columns from comp_final
cleaned_comp = filtered[['gvkey', 'fyear', 'conm', 'Symbol', 'CIK']]

# Drop duplicates from df2_subset
cleaned_comp = cleaned_comp.drop_duplicates()


print('cleaned_comp:',cleaned_comp['gvkey'].nunique())
print('acct_df:',acct_df['gvkey'].nunique())

print('--')


accounting = pd.merge(cleaned_comp,acct_df, how='left',on=['gvkey','fyear'], indicator=True)
print(accounting['gvkey'].nunique())

cleaned_comp: 89
acct_df: 354
--
89


In [13]:
# accounting data
accounting.to_csv("outputs/accounting_final.csv", index = False)

# compustat data
filtered.to_csv("outputs/compustat_final.csv", index = False)